In [1]:
from app.variable.expr import LazyExpr
from typing import List, Dict, Any, Optional
from datetime import datetime
import ast

In [2]:
im = ["import numpy as np", "from datetime import datetime"]

# s = "lambda x: all(isinstance(i, datetime) for i in x) if isinstance(x, list) else False"
s = "(aaaaa := a + 1)"
# s = """
# 1+1
# """
f = LazyExpr(s, im)

# print(ast.dump(f._ast, indent=2))
print(f.unbound)
print(f.target)

f.eval(env={"a": 1024})

{'a'}
aaaaa


1025

In [3]:
from app.data_source.model import APISource

s = APISource(name="test", url="http://localhost:5000/api")
s.model_dump()

C:\Users\cross\AppData\Roaming\Python\Python312\site-packages\pydantic\_internal\_fields.py:186: UserWarning: Field name "schema" shadows an attribute in parent "BaseModel"; 
  warnings.warn(


{'name': 'test',
 'url': 'http://localhost:5000/api',
 'comment': None,
 'schema': None}

In [4]:
from app.variable.model import Variable, LiteralVariable

LiteralVariable(name="s", expr=LazyExpr("1+1"))

LiteralVariable(name='s', expr=LazyExpr(raw='1+1', imports=[]), comment=None, formatter=None, t=None, inst_imports=None)

In [5]:
LiteralVariable.model_validate({
    "name": "s",
    "expr": "1+1"
})


var_a = LiteralVariable.model_validate({
    "name": "a",
    "expr": {
        "raw": "1+1",
        "imports": [],
    },
    "t": "int",
})

var_a.verify()

True

In [11]:
var_s = LiteralVariable(name="s", expr=LazyExpr("datetime.now()"), t=datetime, imports=im)
var_s.imports
var_s.verify()
# var_s.expr

AttributeError: 'LiteralVariable' object has no attribute 'eval'

In [7]:

lst = [
    LiteralVariable(name="s", expr=LazyExpr("1+1")),
    LiteralVariable(name="a", expr=LazyExpr("1+1")),
    LiteralVariable(name="b", expr=LazyExpr("a+1")),
    LiteralVariable(name="c", expr=LazyExpr("b+1")),
    LiteralVariable(name="d", expr=LazyExpr("a+b+2")),
    LiteralVariable(name="e", expr=LazyExpr("c+d+3")),
]
for v in lst:
    print(v.unbound)

set()
set()
{'a'}
{'b'}
{'a', 'b'}
{'d', 'c'}


In [8]:
from app.dependency.resolver import DependencyResolver, resolve_env

resolve_env(lst)

Ok({'a': 2, 'b': 3, 'c': 4, 'd': 7, 'e': 14, 's': 2})

In [9]:
text = "本期中出院病例${{'a':0}}例。手术人数${num_operation}人，四级手术${num_operation_4}人，微创手术${num_operation_micro}人。CMI值${CMI}，其中 CMI 小于1的病人数有${query(CMI<1)}人，占比${query(CMI<1)/num_patient}；1-2的病人数${query(CMI > 1 and CMI < 2)}人，占比${query(CMI > 1 and CMI < 2) / num_patient};2-5的病人数${(num_CIM_2_5 := query(CMI > 2 and CMI < 5))}人，占比${num_CIM_2_5 / num_patient};大于5的病人数${(num_CIM_gt_5 := query(CMI > 5))}人,占比${num_CIM_gt_5 / num_patient}。总体来说大于2的病人数${(num_CIM_gt_2 := query(CMI > 2))}人，占比${num_CIM_gt_2 / num_patient}%，有提升空间。"

In [10]:
from app.dependency.extractor import parse

parsed = parse(text)

# print(parsed)

table = parsed.unwrap().table
exprs = list(map(lambda x: LazyExpr(x[1]), table))
print(list(map(lambda x: x.raw, exprs)))
print(list(map(lambda x: x.target, exprs)))
print(list(map(lambda x: x.unbound, exprs)))

["{'a':0}", 'num_operation', 'num_operation_4', 'num_operation_micro', 'CMI', 'query(CMI<1)', 'query(CMI<1)/num_patient', 'query(CMI > 1 and CMI < 2)', 'query(CMI > 1 and CMI < 2) / num_patient', '(num_CIM_2_5 := query(CMI > 2 and CMI < 5))', 'num_CIM_2_5 / num_patient', '(num_CIM_gt_5 := query(CMI > 5))', 'num_CIM_gt_5 / num_patient', '(num_CIM_gt_2 := query(CMI > 2))', 'num_CIM_gt_2 / num_patient']
[None, None, None, None, None, None, None, None, None, 'num_CIM_2_5', None, 'num_CIM_gt_5', None, 'num_CIM_gt_2', None]
[set(), {'num_operation'}, {'num_operation_4'}, {'num_operation_micro'}, {'CMI'}, {'query', 'CMI'}, {'query', 'CMI', 'num_patient'}, {'query', 'CMI'}, {'query', 'CMI', 'num_patient'}, {'query', 'CMI'}, {'num_CIM_2_5', 'num_patient'}, {'query', 'CMI'}, {'num_CIM_gt_5', 'num_patient'}, {'query', 'CMI'}, {'num_CIM_gt_2', 'num_patient'}]
